In [1]:
import warnings; warnings.simplefilter('ignore')    
import pandas as pd
import numpy as np
# import statsmodels.api as sm
# import scipy.stats as scs
# import tushare as ts
%matplotlib inline
# from scipy.stats import norm
from matplotlib import pyplot as plt
# from scipy.interpolate import PchipInterpolator
# from scipy.integrate import quad
# from scipy import stats
from tqdm import *

## import and process liquidity.csv
### feature: ['Exit price']

In [2]:
liquidity = pd.read_csv('investmentsUK\liquidity.csv')
liquidity = liquidity.drop(columns=['Beauhurst URL','Companies House Number'])    #drop useless information
liquidity = liquidity.rename(columns={"Date of the company's exit or death":"Exit date","Current Stage of Evolution":"Current stage"})
liquidity = liquidity.sort_values(by=['Company name']) #sort the data by company name

In [3]:
liquidity.head()

,Company name,Incorporation date,Current stage,Exit date,Exit price
641,10 Digital Media,NaN,Dead,27/11/2018,0.0
1020,10 Minutes With,19/03/2013,Dead,29/06/2017,0.0
3197,1066 Airsoft,13/05/2014,Dead,27/02/2019,0.0
1076,14M Genomics,09/01/2014,Dead,21/03/2016,0.0
3358,1855,27/02/2013,Dead,20/02/2020,0.0


## import and process deals.csv
### feature: ['Investment amount']

In [4]:
deals = pd.read_csv('investmentsUK\deals.csv')    #import data
deals = deals.drop(columns=['Beauhurst company URL','Beauhurst deal URL','Amount raised (converted to GBP)'])    #drop useless information
deals = deals.rename(columns={'Verified investment amount (converted to GBP)':'Investment amount'})    #rename column
deals = deals.dropna()    #clean missing data rows
deals['Deal date'] = pd.to_datetime(deals['Deal date'],format='%Y-%m-%d')
deals = deals.sort_values(by=['Company name'])    #sort the data by company name

In [5]:
deals.head()

,Company name,Deal date,Investment amount
3866,10 Digital Media,2015-11-16,3398454.0
6303,1066 Airsoft,2016-05-31,23668.0
1395,14M Genomics,2014-12-16,5000000.0
6787,1855,2017-06-30,20000.0
8443,1st Impression,2013-11-22,75005.0


## left merge deals.csv and liquidity.csv to get startups.csv
### feature: ['Investment amount', 'Exit price']

In [6]:
startups = pd.merge(deals, liquidity, on='Company name',how='left')    #left join deals.csv with liquidity.csv which have been processed
startups.loc[(startups['Current stage'] == "Dead") ,'Exit price'] = 0    #put zero in the 'Exit price' column if the startup is dead
startups = startups.dropna(subset=['Exit price'])    #clean data rows whose values in "Exit price" are missing
startups['Exit date'] = pd.to_datetime(startups['Exit date'],format='%d/%m/%Y')
startups['Exit date'] = pd.to_datetime(startups['Exit date'],format='%Y-%m-%d')

In [7]:
startups.head()

,Company name,Deal date,Investment amount,Incorporation date,Current stage,Exit date,Exit price
0,10 Digital Media,2015-11-16,3398454.0,NaN,Dead,2018-11-27,0.0
1,1066 Airsoft,2016-05-31,23668.0,13/05/2014,Dead,2019-02-27,0.0
2,14M Genomics,2014-12-16,5000000.0,09/01/2014,Dead,2016-03-21,0.0
3,1855,2017-06-30,20000.0,27/02/2013,Dead,2020-02-20,0.0
4,1st Impression,2013-11-22,75005.0,26/08/2009,Dead,2019-12-16,0.0


## import and process organizations.csv
### feature: ['first_industry','industry_code']

In [8]:
organizations = pd.read_csv('investmentsUK/organizations.csv')
organizations = organizations.loc[:,['name', 'category_groups_list']]
organizations = organizations.rename(columns={'name':'Company name'})
organizations = organizations.dropna(inplace=False)

In [9]:
organizations['first_industry'] = organizations['category_groups_list'].apply(lambda x:x.split(',')[0])
map_df = pd.DataFrame(organizations['first_industry'].unique()).reset_index().rename(columns={0:'first_industry'})
organizations = pd.merge(organizations,map_df,on='first_industry',how='outer')
organizations = organizations.loc[:,['Company name','first_industry']]
organizations['first_industry'] = organizations['first_industry'].fillna('Other')
organizations = organizations.sort_values(by=['Company name']) 

In [10]:
map_ind = pd.DataFrame(organizations['first_industry'].unique().tolist()).reset_index().rename(columns={0:'first_industry','index':'industry_code'})
organizations = pd.merge(organizations,map_ind,on='first_industry',how='left')

In [11]:
startups = pd.merge(startups,organizations,on='Company name',how='left')

In [12]:
# from sklearn import preprocessing

# data = (organizations.category_groups_list.str.split('\s*,\s*', expand=True)
#    .stack()
#    .str.get_dummies()
#    .sum(level=0))

# enc = preprocessing.OneHotEncoder(sparse=False)
# data = enc.fit_transform( organizations['first_industry'].values.reshape(-1,1) )

# organizations['one_hot'] = list(data)

## import and process funding_rounds.csv to get founder_count.csv
### feature: ['investor_count']

In [13]:
founder_count = pd.read_csv('investmentsUK\\funding_rounds.csv')
funds = founder_count
founder_count = founder_count.loc[:,['org_name','investor_count']]
founder_count = founder_count.rename(columns={'org_name':'Company name'}) 
founder_count = founder_count.dropna(inplace=False)
founder_count = founder_count.drop_duplicates(subset=None, keep='first', inplace=False)
founder_count = founder_count.sort_values(by=['Company name']) 

In [14]:
founder_count.head()

,Company name,investor_count
116667,!hey software,2.0
33185,#Vine,2.0
341803,#Yolo ex,1.0
106036,#hexology,1.0
266857,#paid,1.0


## left merge startups.csv and founder_count.csv
### feature: ['Investment amount', 'Exit price', 'founder_count']

In [15]:
startups = pd.merge(startups, founder_count, on='Company name',how='left') 

In [16]:
startups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7901 entries, 0 to 7900
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        7901 non-null   object        
 1   Deal date           7901 non-null   datetime64[ns]
 2   Investment amount   7901 non-null   float64       
 3   Incorporation date  7723 non-null   object        
 4   Current stage       7901 non-null   object        
 5   Exit date           7901 non-null   datetime64[ns]
 6   Exit price          7901 non-null   float64       
 7   first_industry      4838 non-null   object        
 8   industry_code       4838 non-null   float64       
 9   investor_count      3879 non-null   float64       
dtypes: datetime64[ns](2), float64(4), object(4)
memory usage: 679.0+ KB


## import funding_rounds.csv as 'funds' to filter the valuation of later-stage companies
### feature: ['mean_val']

In [17]:
funds = funds.loc[:,['org_name', 'investment_type','raised_amount_usd']]
funds = funds[funds['investment_type'].isin(['series_c','series_d'])] #filter the later stage investments
funds = funds.rename(columns={'org_name':'Company name'})
funds['raised_amount_gbp'] = funds['raised_amount_usd']*0.72
funds = funds.drop(columns=['raised_amount_usd'])
funds = funds.dropna(inplace=False)

## left merge organizations.csv and funds.csv 

In [18]:
ind_funds = pd.merge(organizations, funds, on='Company name',how='left') 
# ind_funds = ind_funds.drop(columns='index')
ind_funds = ind_funds.dropna()
ind_funds = ind_funds.sort_values(by=['first_industry','raised_amount_gbp'])

## detect outliers

In [19]:
industry = ind_funds['first_industry'].unique()
u = []
std = []
for i in tqdm(industry):
    u.append(ind_funds['raised_amount_gbp'].loc[ind_funds['first_industry']==i].mean())
    std.append(ind_funds['raised_amount_gbp'].loc[ind_funds['first_industry']==i].std())

100%|█████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 233.29it/s]


In [20]:
data = {'first_industry':industry,'mean':u,'std':std}
u_std = pd.DataFrame(data)
ind_val = pd.merge(ind_funds,u_std,on='first_industry',how='left')
ind_val['bias'] = (ind_val['raised_amount_gbp']-ind_val['mean']).abs()
ind_val['error']=0
ind_val.loc[ind_val['bias']>(ind_val['std']*3),'error'] = 1

## remove outliers

In [21]:
ind_norm = ind_val.loc[ind_val['error']==0]

In [22]:
# plt.style.use({'figure.figsize':(4, 2.5)}) #visualize the valuation distribution
# # fig,axes=plt.subplots(11,4)
# # for m in range(0,41):
# for i in industry :
#     sns.distplot(ind_norm['raised_amount_gbp'].loc[ind_norm['first_industry']==i],label=i)
#     plt.show()

## mean valuation of various industries

In [23]:
mean=[]
for i in industry:
    mean.append(ind_norm['raised_amount_gbp'].loc[ind_norm['first_industry']==i].mean())

In [24]:
data = {'first_industry':industry,'mean_val':mean}
ind_val = pd.DataFrame(data)

## left startups.csv and ind_val.csv

In [25]:
startups['first_industry'] = startups['first_industry'].fillna('Other')
startups = pd.merge(startups,ind_val,on=['first_industry'],how='left')
startups['industry_level'] = startups['Investment amount'] - startups['mean_val']

## import and process jobs.csv & people.csv to get founder_gender.csv
### feature: ['male', 'female']

In [26]:
jobs = pd.read_csv('investmentsUK/jobs.csv') 
jobs = jobs.loc[:,['person_name', 'org_name', 'title']]
jobs = jobs.rename(columns={'person_name':'founder_name'})
jobs = jobs.rename(columns={'org_name':'Company name'})
jobs = jobs.dropna(inplace=False)
searchfor = ['found','Found']
jobs = jobs[jobs['title'].str.contains('|'.join(searchfor))] 
#search for 'founder'/'co-founder'/'founding partner'
Cname = startups[['Company name']]
jobs = pd.merge(Cname,jobs,on='Company name',how='left')
jobs = jobs.dropna(subset=['founder_name','title'],inplace=False)
jobs = jobs.sort_values(by=['Company name']) 

In [27]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10737 entries, 12 to 14927
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company name  10737 non-null  object
 1   founder_name  10737 non-null  object
 2   title         10737 non-null  object
dtypes: object(3)
memory usage: 335.5+ KB


In [28]:
people = pd.read_csv('investmentsUK/people.csv') 
people = people.loc[:,['name', 'gender']]
people = people.rename(columns={'name':'founder_name'})
founder = pd.merge(jobs,people,on='founder_name',how='left')
founder = founder.drop_duplicates(subset=None, keep='first', inplace=False) 

In [29]:
founder.head()

,Company name,founder_name,title,gender
0,3 Kinds of Ice,David Bailey,CEO & Co-Founder,male
19,3sun Group,Willie Dawson,Founder,male
20,4x4 Aviation,Thorsten Reinhardt,Founder & CEO,male
21,A Suit That Fits,Warren Bennett,Co-Founder,male
25,ACAL Energy,Andrew Creeth,Chief Technology Officer & Co-Founder,male


In [30]:
length_column = founder.shape[0]
founder['male'] = (0)
founder['female'] = (0)
founder['founder_count'] = (0)
for i in tqdm(range(0,length_column)):
    if (founder['Company name'].iloc[i] == founder['Company name'].iloc[i-1])&(founder['gender'].iloc[i] == 'male'):
        founder['male'].iloc[i] = founder['male'].iloc[i-1]+1
        founder['founder_count'].iloc[i] += 1
    elif (founder['Company name'].iloc[i] == founder['Company name'].iloc[i-1])&(founder['gender'].iloc[i] == 'female'):
        founder['female'].iloc[i] = founder['female'].iloc[i-1]+1
        founder['founder_count'].iloc[i] += 1
    elif founder['gender'].iloc[i] == 'male':
        founder['male'].iloc[i] = 1
    else:
        founder['female'].iloc[i] = 1
founder_gender = pd.concat([founder,founder.fillna(0)],axis=0)
founder_gender = founder_gender.drop_duplicates(subset=['Company name'], keep='last', inplace=False)
founder_gender = founder_gender.loc[:,['Company name','male','female','founder_count']]

100%|████████████████████████████████████████████████████████████████████████████| 1369/1369 [00:01<00:00, 1164.11it/s]


In [31]:
founder_gender

,Company name,male,female,founder_count
0,3 Kinds of Ice,1,0,0
19,3sun Group,1,0,0
20,4x4 Aviation,1,0,0
21,A Suit That Fits,1,0,0
25,ACAL Energy,1,0,0
...,...,...,...,...
18274,passingboxes,0,1,0
18276,people.io,1,0,0
18280,rtobjects,1,0,0
18286,uberVU,3,0,1


## left merge startups.csv and founder_count.csv
### feature: ['Investment amount', 'Exit price', 'founder_count', 'male', 'female']

In [32]:
startups = pd.merge(startups, founder_gender, on='Company name',how='left')
startups = startups.drop_duplicates(subset=['Company name','Deal date'], keep='first', inplace=False) 

In [33]:
startups.head()

,Company name,Deal date,Investment amount,Incorporation date,Current stage,Exit date,Exit price,first_industry,industry_code,investor_count,mean_val,industry_level,male,female,founder_count
0,10 Digital Media,2015-11-16,3398454.0,NaN,Dead,2018-11-27,0.0,Other,NaN,NaN,2.535893e+07,-2.196048e+07,NaN,NaN,NaN
1,1066 Airsoft,2016-05-31,23668.0,13/05/2014,Dead,2019-02-27,0.0,Other,NaN,NaN,2.535893e+07,-2.533526e+07,NaN,NaN,NaN
2,14M Genomics,2014-12-16,5000000.0,09/01/2014,Dead,2016-03-21,0.0,Biotechnology,29.0,NaN,2.088792e+07,-1.588792e+07,NaN,NaN,NaN
3,1855,2017-06-30,20000.0,27/02/2013,Dead,2020-02-20,0.0,Other,NaN,NaN,2.535893e+07,-2.533893e+07,NaN,NaN,NaN
4,1st Impression,2013-11-22,75005.0,26/08/2009,Dead,2019-12-16,0.0,Other,NaN,NaN,2.535893e+07,-2.528392e+07,NaN,NaN,NaN


## import and process shares.csv
### feature: ['number_of_shares', 'percent_total']

In [34]:
shares = pd.read_csv('investmentsUK\shares.csv')    #import data
shares = shares.loc[:,['Company name','filing_date', 'first_name', 'last_name', 'number_of_shares','percent_total']]    #select and retain useful information
shares = shares.dropna(subset=['last_name'])  
# shares = shares[~shares['number_of_shares'].isin([0])]    #clean data rows whose values in "number_of_shares" or "last_name" are missing
shares['first_name'] = shares['first_name'].fillna('')    #replace Nan to "" in "first_name" column
shares = shares.drop_duplicates(subset=None, keep='first', inplace=False)    #eliminate duplicate data rows
shares['filing_date'] = pd.to_datetime(shares['filing_date'],format='%d/%m/%Y')
shares['filing_date'] = pd.to_datetime(shares['filing_date'],format='%Y-%m-%d')
shares = shares.sort_values(by=['Company name','filing_date','last_name', 'number_of_shares'])    
#set Investor name as index and sort the data first by 'Company name' then by 'number_of_shares'
# investors = shares.drop_duplicates(subset=['Company name','number_of_shares'], keep='first', inplace=False)
#eliminate duplicate data rows whose number_of_shares make no change [179719 rows x 6 columns] --> [87229 rows x 6 columns]

In [35]:
shares.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179398 entries, 33100 to 85938
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Company name      179398 non-null  object        
 1   filing_date       179398 non-null  datetime64[ns]
 2   first_name        179398 non-null  object        
 3   last_name         179398 non-null  object        
 4   number_of_shares  179398 non-null  float64       
 5   percent_total     179398 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 9.6+ MB


## Calculate how many shares have been issued each round 
### feature: ['round', 'round_shares']

In [36]:
rounds = shares.drop_duplicates(subset=['filing_date','Company name'], keep='last') 
rounds['total_shares'] = rounds['number_of_shares']*100/rounds['percent_total'] 
rounds = rounds.dropna(subset=['total_shares'])
#Calculate the cumulative number of shares issued by the startup firms in each funding round
# rounds.round({'total_shares': 2})
rounds = rounds.loc[:,['Company name','filing_date','total_shares']]
length_column = rounds.shape[0]
rounds['round_shares'] = ''
for i in tqdm(range(0,length_column)):
    if rounds['Company name'].iloc[i] == rounds['Company name'].iloc[i-1]:
        rounds['round_shares'].iloc[i] = rounds['total_shares'].iloc[i]-rounds['total_shares'].iloc[i-1]
    else:
        rounds['round_shares'].iloc[i] = rounds['total_shares'].iloc[i]
#Calculate the number of shares issued separately by the startup in each funding round

100%|██████████████████████████████████████████████████████████████████████████| 14982/14982 [00:07<00:00, 1955.05it/s]


In [37]:
rounds = rounds[~rounds['round_shares'].isin([0])] 
# if the value of round_shares is zero, there was not a new funding round
length_column = rounds.shape[0]
rounds['rounds'] = ''
for i in tqdm(range(0,length_column)):
    if rounds['Company name'].iloc[i] == rounds['Company name'].iloc[i-1]:
        rounds['rounds'].iloc[i] = rounds['rounds'].iloc[i-1]+1
    else:
        rounds['rounds'].iloc[i] = 1

100%|████████████████████████████████████████████████████████████████████████████| 8638/8638 [00:04<00:00, 2075.28it/s]


In [38]:
# company_name_lis = rounds['Company name'].unique().tolist()
# for cmp in tqdm(company_name_lis):
#     dd = rounds[rounds['Company name'] == cmp]
#     length_column = dd.shape[0]
#     dd['round_share'] = dd['number_of_shares'] - dd['number_of_shares'].shift()
#     dd['round'] = length_column
#     rounds = pd.concat([rounds,dd.fillna(0)],axis=0)

In [39]:
rounds

,Company name,filing_date,total_shares,round_shares,rounds
33100,10 Digital Media,2015-06-18,1.000000e+02,100.0,1
52887,10 Minutes With,2013-03-19,1.000000e+06,1000000.0,1
52891,10 Minutes With,2014-03-19,2.577320e+06,1577319.587629,2
52919,10 Minutes With,2015-03-19,3.048780e+06,471460.900176,3
52978,10 Minutes With,2016-03-19,3.244444e+06,195663.95664,4
...,...,...,...,...,...
136317,yWasteFood,2015-11-27,1.000000e+01,10.0,1
136318,yWasteFood,2016-11-26,1.399992e+01,3.999916,2
85764,âcasă,2017-07-19,1.628879e+06,1628878.894957,1
85804,âcasă,2018-07-19,2.411378e+06,782498.996066,2


## Select those start-ups who have raised fundings for 4 times

In [40]:
# startups = startups.drop_duplicates(subset=['Company name','Deal date'])
count_value=startups['Company name'].value_counts()
limit = list(count_value[count_value>3].index)
startups_limit = startups[startups['Company name'].isin(limit)]

In [41]:
count_value=rounds['Company name'].value_counts()
limit = list(count_value[count_value==4].index)
rounds_limit = rounds[rounds['Company name'].isin(limit)]
shares_limit = pd.merge(rounds_limit,shares, on=['Company name','filing_date'],how='left')

In [42]:
shares_limit['Company name'].value_counts()

The London Distillery Company    395
Wonga                            352
Ecco Recordings                  350
Neyber                           323
Metalysis                        310
                                ... 
IPS                                7
Tiller                             4
CN Creative Group                  4
IOmet Pharma                       4
Oxitec                             4
Name: Company name, Length: 325, dtype: int64

In [43]:
startups_limit['Company name'].value_counts()
# startups_limit.head()

Houseology                  13
NetThings                   12
Microtest Matrices          11
Streetlife                  11
Critical Pharmaceuticals     9
                            ..
Deals & Meals                4
Dead Right                   4
Thoughtonomy                 4
Ongosa                       4
Novolytics                   4
Name: Company name, Length: 321, dtype: int64

## Inner merge startups.csv and shares_limit.csv

In [44]:
df = pd.merge(shares_limit,startups_limit,on='Company name',how='inner')
# df = df[['Company name','Deal date','Investment amount','filing_date','Investor name','number_of_shares','percent_total','Current stage','Incorporation date','Exit date','Exit price']]
df = df.sort_values(by=['Company name','last_name','Deal date','filing_date']) 
df=df.loc[(df['Deal date']<df['filing_date'] )]
# drop deals whose deal date cannot match with filing date
df=df.drop_duplicates(subset=['filing_date','first_name','last_name','Company name'], keep='last') 
#match 'filing_date' with the closest 'Deal date'

In [45]:
# df['Company name'].value_counts()
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8445 entries, 4 to 51433
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        8445 non-null   object        
 1   filing_date         8445 non-null   datetime64[ns]
 2   total_shares        8445 non-null   float64       
 3   round_shares        8445 non-null   object        
 4   rounds              8445 non-null   object        
 5   first_name          8445 non-null   object        
 6   last_name           8445 non-null   object        
 7   number_of_shares    8445 non-null   float64       
 8   percent_total       8445 non-null   float64       
 9   Deal date           8445 non-null   datetime64[ns]
 10  Investment amount   8445 non-null   float64       
 11  Incorporation date  8241 non-null   object        
 12  Current stage       8445 non-null   object        
 13  Exit date           8445 non-null   datetime64[

## Calculate other features

In [46]:
exit_share = rounds.loc[:,['Company name','filing_date','total_shares']]
exit_share = exit_share.drop_duplicates(subset='Company name', keep='last')
exit_share = exit_share.rename(columns={'total_shares':'exit_shares'})
#Calculate the cumulative number of shares issued by the company at the time of investor exit
exit_share = exit_share.drop(columns='filing_date')
df = pd.merge(df, exit_share, on='Company name',how='left')

In [47]:
length_column = df.shape[0]
df['add_shares'] = ''
for i in tqdm(range(0,length_column)):
    if (df['last_name'].iloc[i] == df['last_name'].iloc[i-1])&(df['first_name'].iloc[i] == df['first_name'].iloc[i-1]):
        df['add_shares'].iloc[i] = df['number_of_shares'].iloc[i]-df['number_of_shares'].iloc[i-1]
    else:
        df['add_shares'].iloc[i] = df['number_of_shares'].iloc[i]

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:04<00:00, 1865.42it/s]


In [48]:
df['follow'] = (0)
df.loc[(df['add_shares']>0) ,'follow'] = 1 #set the follow-on decisions as the label 

In [49]:
df['valuation'] = df['Investment amount']/df['round_shares']

### Calculate Step_up Valuation

In [50]:
step_up = df.loc[:,['Company name','filing_date','rounds','valuation']]
step_up = step_up.drop_duplicates(subset=['Company name','filing_date']).sort_values(by=['Company name','filing_date'])
length_column = step_up.shape[0]
step_up['value_chg'] = (0)
for i in tqdm(range(0,length_column)):
    if (step_up['Company name'].iloc[i] == step_up['Company name'].iloc[i-1]):
        step_up['value_chg'].iloc[i] = step_up['valuation'].iloc[i]/step_up['valuation'].iloc[i-1]*step_up['value_chg'].iloc[i-1]
    else:
        step_up['value_chg'].iloc[i] = 1

100%|██████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<00:00, 1873.10it/s]


In [51]:
# step_up['v2/v1'] = ''
# step_up['v3/v2'] = ''
# step_up['v4/v3'] = ''
# step_up.loc[(step_up['rounds']==2),'v2/v1']=step_up['value_chg']
# step_up.loc[(step_up['rounds']==3),'v3/v2']=step_up['value_chg']
# step_up.loc[(step_up['rounds']==3),'v2/v1']=step_up['v2/v1'].shift()
# step_up.loc[(step_up['rounds']==4),'v4/v3']=step_up['value_chg']
# step_up.loc[(step_up['rounds']==4),'v3/v2']=step_up['v3/v2'].shift()
# step_up.loc[(step_up['rounds']==4),'v2/v1']=step_up['v2/v1'].shift()
step_up = step_up.loc[:,['Company name','rounds','value_chg']]

In [52]:
step_up.head()

,Company name,rounds,value_chg
0,ATEEDA,1,1.000000
1,ATEEDA,2,2.395736
2,ATEEDA,3,-16.043576
3,ATEEDA,4,1.985387
261,Adbrain,1,1.000000


In [53]:
df = pd.merge(df,step_up,on=['Company name','rounds'],how='left')
df = df.drop_duplicates(subset=['Company name','last_name','first_name','filing_date'])

In [54]:
df = df.sort_values(by=['Company name','last_name','rounds'])
length_column = df.shape[0]
df['follow_count'] = (0)
for i in tqdm(range(0,length_column)):
    if (df['last_name'].iloc[i] == df['last_name'].iloc[i-1])&(df['first_name'].iloc[i] == df['first_name'].iloc[i-1])&(df['Company name'].iloc[i] == df['Company name'].iloc[i-1]):
        df['follow_count'].iloc[i] = df['follow'].iloc[i]+df['follow_count'].iloc[i-1]

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:02<00:00, 3648.93it/s]


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8445 entries, 0 to 8444
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        8445 non-null   object        
 1   filing_date         8445 non-null   datetime64[ns]
 2   total_shares        8445 non-null   float64       
 3   round_shares        8445 non-null   object        
 4   rounds              8445 non-null   object        
 5   first_name          8445 non-null   object        
 6   last_name           8445 non-null   object        
 7   number_of_shares    8445 non-null   float64       
 8   percent_total       8445 non-null   float64       
 9   Deal date           8445 non-null   datetime64[ns]
 10  Investment amount   8445 non-null   float64       
 11  Incorporation date  8241 non-null   object        
 12  Current stage       8445 non-null   object        
 13  Exit date           8445 non-null   datetime64[n

## Calculate the number of follow-on investors in the last rounds

In [56]:
rate = df.sort_values(by=['Company name','rounds'])
rate = rate.drop_duplicates()
# rate['Company name'].unique()

In [57]:
rate = rate.loc[:,['Company name','rounds','follow']]
length_column = rate.shape[0]
rate['num'] = 0
rate['fo'] = 0
for i in tqdm(range(0,length_column)):
    if rate['rounds'].iloc[i]==rate['rounds'].iloc[i-1]:
        rate['num'].iloc[i] = rate['num'].iloc[i-1]+1
        rate['fo'].iloc[i] = rate['fo'].iloc[i-1]+rate['follow'].iloc[i]
    else:
        rate['num'].iloc[i] = 1
        rate['fo'].iloc[i] = rate['follow'].iloc[i]

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:07<00:00, 1092.76it/s]


In [58]:
rate = rate.drop_duplicates(subset=['Company name','rounds'],keep='last')
rate['rate'] = round(rate['fo']/rate['num'],2)
rate = rate.loc[:,['Company name','rounds','rate']]
rate = rate.loc[rate['rounds']>1]
rate['rounds'] = rate['rounds']+1
df = pd.merge(df,rate,on=['Company name','rounds'],how='left')

In [59]:
df.to_csv(r'df.csv')  

## Calculate TVPI(total value to paid-in)

In [60]:
df_tvpi = df.sort_values(by=['Company name','last_name','rounds'])
length_column = df_tvpi.shape[0]
df_tvpi['paid_in'] = ''
df_tvpi['follow_count'] = (0)
for i in tqdm(range(0,length_column)):
    if (df_tvpi['last_name'].iloc[i] == df_tvpi['last_name'].iloc[i-1])&(df_tvpi['first_name'].iloc[i] == df_tvpi['first_name'].iloc[i-1])&(df_tvpi['Company name'].iloc[i] == df_tvpi['Company name'].iloc[i-1]):
        df_tvpi['paid_in'].iloc[i] = df_tvpi['valuation'].iloc[i]*df_tvpi['add_shares'].iloc[i]+df_tvpi['paid_in'].iloc[i-1]
        df_tvpi['follow_count'].iloc[i] = df_tvpi['follow'].iloc[i]+df_tvpi['follow_count'].iloc[i-1]
    else:
        df_tvpi['paid_in'].iloc[i] = df_tvpi['valuation'].iloc[i]*df_tvpi['add_shares'].iloc[i]
#calculate the cumulative paid-in of investors

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:06<00:00, 1324.70it/s]


In [61]:
df_tvpi.loc[df_tvpi['last_name']=='VYZAS']

,Company name,filing_date,total_shares,round_shares,rounds,first_name,last_name,number_of_shares,percent_total,Deal date,...,female,founder_count,exit_shares,add_shares,follow,valuation,value_chg,follow_count,rate,paid_in
7818,Upstream,2015-06-13,146053.853296,146053.853296,1,ELIAS,VYZAS,1573.0,1.077,2015-04-30,...,0.0,1.0,208344.370861,1573.0,1,9.229048,1.000000,0,NaN,14517.29226
7819,Upstream,2016-06-12,163005.181347,16951.328051,2,ELIAS,VYZAS,1573.0,0.965,2016-06-01,...,0.0,1.0,208344.370861,0.0,0,41.907985,4.540878,0,NaN,14517.29226
7820,Upstream,2017-06-12,173428.886439,10423.705092,3,ELIAS,VYZAS,1573.0,0.907,2017-03-09,...,0.0,1.0,208344.370861,0.0,0,19.183774,2.078630,0,0.25,14517.29226
7821,Upstream,2018-06-12,208344.370861,34915.484422,4,ELIAS,VYZAS,1573.0,0.755,2017-06-22,...,0.0,1.0,208344.370861,0.0,0,16.167469,1.751802,0,0.50,14517.29226


In [62]:
df_tvpi = df_tvpi.drop_duplicates(subset=['Company name','first_name','last_name'], keep='last')
df_tvpi['total_value'] = df_tvpi['Exit price']/df_tvpi['exit_shares']*df_tvpi['number_of_shares']
df_tvpi['TVPI'] = df_tvpi['total_value']/df_tvpi['paid_in']

In [63]:
df_tvpi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3613 entries, 3 to 8444
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        3613 non-null   object        
 1   filing_date         3613 non-null   datetime64[ns]
 2   total_shares        3613 non-null   float64       
 3   round_shares        3613 non-null   object        
 4   rounds              3613 non-null   object        
 5   first_name          3613 non-null   object        
 6   last_name           3613 non-null   object        
 7   number_of_shares    3613 non-null   float64       
 8   percent_total       3613 non-null   float64       
 9   Deal date           3613 non-null   datetime64[ns]
 10  Investment amount   3613 non-null   float64       
 11  Incorporation date  3539 non-null   object        
 12  Current stage       3613 non-null   object        
 13  Exit date           3613 non-null   datetime64[n

In [64]:
df_tvpi.to_csv(r'df_tvpi.csv')  

In [65]:
mean_TVPI = np.mean(df_tvpi['TVPI'].values.tolist())
mean_TVPI

0.13248259236984766

## Add features: 'rate', 'first_industry', 'industry_level'

In [66]:
# rate = pd.read_csv('rate.csv')
# del rate['Unnamed: 0']
# section_limit = pd.read_csv('section_limit.csv')
# del section_limit['Unnamed: 0']
# ind_val = pd.read_csv('ind_val.csv')
# del ind_val['Unnamed: 0']

In [67]:
# df1 = pd.merge(df,rate,on=['Company name','rounds'],how='left')
# df1['rate'] = df1['rate'].fillna(1)
# df2 = pd.merge(df1,section_limit,on=['first_industry','rounds'],how='left')
# df2['first_industry'] = df2['first_industry'].fillna('Other')
# df3 = pd.merge(df2,ind_val,on=['first_industry'],how='left')
# df3['industry_level'] = df3['Investment amount'] - df3['mean_val']

In [68]:
# map_ind = pd.DataFrame(df3['first_industry'].unique().tolist()).reset_index().rename(columns={0:'first_industry','index':'industry_code'})
# df3 = pd.merge(df3,map_ind,on='first_industry',how='left')

## Fill null data

In [69]:
df['male_ratio'] = df['male']/(df['male']+df['female'])
df['male_ratio'] = df['male_ratio'].fillna(0.5)

In [70]:
df['label'] = df['follow'].astype(int) 

In [71]:
df['investor_count'] = df['investor_count'].fillna(df['investor_count'].mean())
df['founder_count'] = df['founder_count'].fillna(df['founder_count'].mean())

In [72]:
# df['value_chg'] = (df['value_chg'] - df['value_chg'].min()) / (df['value_chg'].max() - df['value_chg'].min())

In [73]:
# df['percent_total'] = (df['percent_total'] - df['percent_total'].min()) / (df['percent_total'].max() - df['percent_total'].min())

In [74]:
df.to_csv('training_data.csv')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8445 entries, 0 to 8444
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        8445 non-null   object        
 1   filing_date         8445 non-null   datetime64[ns]
 2   total_shares        8445 non-null   float64       
 3   round_shares        8445 non-null   object        
 4   rounds              8445 non-null   object        
 5   first_name          8445 non-null   object        
 6   last_name           8445 non-null   object        
 7   number_of_shares    8445 non-null   float64       
 8   percent_total       8445 non-null   float64       
 9   Deal date           8445 non-null   datetime64[ns]
 10  Investment amount   8445 non-null   float64       
 11  Incorporation date  8241 non-null   object        
 12  Current stage       8445 non-null   object        
 13  Exit date           8445 non-null   datetime64[n